In this notebook you will learn how to do some basic statistical analyses on ELISA data downloaded from ImmPort, where IgG, IgA, and IgM antibody values were measured in subjects longitudinally. We are interested in understanding how these antibody values change over time across subjects, and how these values might change over time based on other clinical/demographic information. 

In [ ]:
#This cell sets everything up, please run this first
#Click on the cells to reveal the code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_excel('ELISATUTPROJ.xlsx')

In [ ]:
data.head()

In [ ]:
# Use the 'Age' column for the age distribution
filtered_data = data[['subjectAccession', 'armName', 'race', 'ethnicity', 'gender', 'Age']].drop_duplicates(subset='subjectAccession')

# Count unique subjects under different categories
armName_distribution = filtered_data['armName'].value_counts()
race_distribution = filtered_data['race'].value_counts()
ethnicity_distribution = filtered_data['ethnicity'].value_counts()
gender_distribution = filtered_data['gender'].value_counts()

# Extract age column for age distribution
age_distribution = filtered_data['Age'].dropna().astype(int)

# Plot distributions
fig, axes = plt.subplots(5, 1, figsize=(12, 25))

axes[0].bar(armName_distribution.index, armName_distribution.values)
axes[0].set_title('Distribution of Subjects by armName')
axes[0].set_xlabel('armName')
axes[0].set_ylabel('Number of Subjects')

axes[1].bar(race_distribution.index, race_distribution.values)
axes[1].set_title('Distribution of Subjects by Race')
axes[1].set_xlabel('Race')
axes[1].set_ylabel('Number of Subjects')

axes[2].bar(ethnicity_distribution.index, ethnicity_distribution.values)
axes[2].set_title('Distribution of Subjects by Ethnicity')
axes[2].set_xlabel('Ethnicity')
axes[2].set_ylabel('Number of Subjects')

axes[3].bar(gender_distribution.index, gender_distribution.values)
axes[3].set_title('Distribution of Subjects by Gender')
axes[3].set_xlabel('Gender')
axes[3].set_ylabel('Number of Subjects')

axes[4].hist(age_distribution, bins=20)
axes[4].set_title('Distribution of Subjects by Age')
axes[4].set_xlabel('Age')
axes[4].set_ylabel('Number of Subjects')

plt.tight_layout()
plt.show()


In [ ]:
# Filter necessary columns
analyte_data = data[['subjectAccession', 'studyTimeCollectedDays', 'analyteReported', 'valueReported']]

# Filter for the analytes IgA, IgG, and IgM
analytes_of_interest = ['IgA', 'IgG', 'IgM']
analyte_data_filtered = analyte_data[analyte_data['analyteReported'].isin(analytes_of_interest)]

# Convert 'studyTimeCollectedDays' to a numeric type
analyte_data_filtered['studyTimeCollectedDays'] = pd.to_numeric(analyte_data_filtered['studyTimeCollectedDays'], errors='coerce')

import seaborn as sns

# Plot box plots for each analyte
plt.figure(figsize=(18, 12))

for i, analyte in enumerate(analytes_of_interest, 1):
    plt.subplot(3, 1, i)
    sns.boxplot(x='studyTimeCollectedDays', y='valueReported', data=analyte_data_filtered[analyte_data_filtered['analyteReported'] == analyte])
    plt.title(f'{analyte} Values Over Time')
    plt.xlabel('Days')
    plt.ylabel('Value Reported')

plt.tight_layout()
plt.show()


In [ ]:
# Merge analyte data with the filtered data to include 'armName' information
merged_data = pd.merge(analyte_data_filtered, filtered_data[['subjectAccession', 'armName']], on='subjectAccession', how='left')

# Plot box plots for each analyte showing the difference between 'armName' categories on each day
plt.figure(figsize=(18, 18))

for i, analyte in enumerate(analytes_of_interest, 1):
    plt.subplot(3, 1, i)
    sns.boxplot(x='studyTimeCollectedDays', y='valueReported', hue='armName', data=merged_data[merged_data['analyteReported'] == analyte])
    plt.title(f'{analyte} Values Over Time by armName')
    plt.xlabel('Days')
    plt.ylabel('Value Reported')
    plt.legend(title='armName')

plt.tight_layout()
plt.show()


In [ ]:
from scipy.stats import f_oneway

# Prepare data for ANOVA
anova_results = {}

for analyte in analytes_of_interest:
    analyte_data = merged_data[merged_data['analyteReported'] == analyte]
    grouped_data = analyte_data.groupby('studyTimeCollectedDays')['valueReported'].apply(list)
    
    # Run ANOVA
    anova_result = f_oneway(*grouped_data)
    anova_results[analyte] = anova_result

anova_results


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Function to perform Tukey's HSD test
def tukey_hsd_test(data, analyte):
    analyte_data = data[data['analyteReported'] == analyte]
    tukey_result = pairwise_tukeyhsd(endog=analyte_data['valueReported'],
                                     groups=analyte_data['studyTimeCollectedDays'],
                                     alpha=0.05)
    return tukey_result

# Perform Tukey's HSD test for IgG and IgM
tukey_result_IgG = tukey_hsd_test(merged_data, 'IgG')
tukey_result_IgM = tukey_hsd_test(merged_data, 'IgM')

tukey_result_IgG.summary()


In [ ]:
tukey_result_IgM.summary()

In [ ]:
import seaborn as sns

# Pivot the data for heatmap creation
def create_heatmap_data(data, analyte):
    pivot_data = data[data['analyteReported'] == analyte].pivot_table(
        values='valueReported', 
        index='studyTimeCollectedDays', 
        columns='armName', 
        aggfunc='mean'
    )
    return pivot_data

# Create heatmaps
fig, axes = plt.subplots(3, 1, figsize=(12, 18))

for i, analyte in enumerate(analytes_of_interest):
    heatmap_data = create_heatmap_data(merged_data, analyte)
    sns.heatmap(heatmap_data, ax=axes[i], cmap='viridis', cbar=True)
    axes[i].set_title(f'Heatmap of {analyte} Values by armName Over Time')
    axes[i].set_xlabel('armName')
    axes[i].set_ylabel('Days')

plt.tight_layout()
plt.show()


In [ ]:
# Merge analyte data with the filtered data to include 'gender' information
merged_data = pd.merge(analyte_data_filtered, filtered_data[['subjectAccession', 'armName', 'gender']], on='subjectAccession', how='left')
# Function to create heatmap data by gender
def create_heatmap_data_by_gender(data, analyte, gender):
    pivot_data = data[(data['analyteReported'] == analyte) & (data['gender'] == gender)].pivot_table(
        values='valueReported', 
        index='studyTimeCollectedDays', 
        columns='armName', 
        aggfunc='mean'
    )
    return pivot_data

genders = ['Male', 'Female']

# Create heatmaps for each gender
fig, axes = plt.subplots(3, 2, figsize=(24, 18))

for i, analyte in enumerate(analytes_of_interest):
    for j, gender in enumerate(genders):
        heatmap_data = create_heatmap_data_by_gender(merged_data, analyte, gender)
        sns.heatmap(heatmap_data, ax=axes[i, j], cmap='viridis', cbar=True)
        axes[i, j].set_title(f'Heatmap of {analyte} Values by armName Over Time ({gender})')
        axes[i, j].set_xlabel('armName')
        axes[i, j].set_ylabel('Days')

plt.tight_layout()
plt.show()

# Prepare data for ANOVA by gender
anova_results_gender = {}

for analyte in analytes_of_interest:
    for gender in genders:
        analyte_data = merged_data[(merged_data['analyteReported'] == analyte) & (merged_data['gender'] == gender)]
        grouped_data = analyte_data.groupby('studyTimeCollectedDays')['valueReported'].apply(list)
        
        # Run ANOVA
        anova_result = f_oneway(*grouped_data)
        anova_results_gender[(analyte, gender)] = anova_result

anova_results_gender


In [ ]:
# Perform Tukey's HSD test for IgM in males
tukey_result_IgM_male = tukey_hsd_test(merged_data[(merged_data['analyteReported'] == 'IgM') & (merged_data['gender'] == 'Male')], 'IgM')
tukey_result_IgM_male.summary()


In [ ]:
# Merge analyte data with the filtered data to include 'race' information
merged_data = pd.merge(analyte_data_filtered, filtered_data[['subjectAccession', 'armName', 'gender', 'race']], on='subjectAccession', how='left')
# Define races
races = merged_data['race'].unique()

# Create heatmaps for each race
fig, axes = plt.subplots(3, len(races), figsize=(24, 18))

for i, analyte in enumerate(analytes_of_interest):
    for j, race in enumerate(races):
        heatmap_data = merged_data[(merged_data['analyteReported'] == analyte) & (merged_data['race'] == race)].pivot_table(
            values='valueReported', 
            index='studyTimeCollectedDays', 
            columns='armName', 
            aggfunc='mean'
        )
        sns.heatmap(heatmap_data, ax=axes[i, j], cmap='viridis', cbar=True)
        axes[i, j].set_title(f'{analyte} - {race}')
        axes[i, j].set_xlabel('armName')
        axes[i, j].set_ylabel('Days')

plt.tight_layout()
plt.show()

# Prepare data for ANOVA by race
anova_results_race = {}

for analyte in analytes_of_interest:
    for race in races:
        analyte_data = merged_data[(merged_data['analyteReported'] == analyte) & (merged_data['race'] == race)]
        grouped_data = analyte_data.groupby('studyTimeCollectedDays')['valueReported'].apply(list)
        
        # Run ANOVA
        anova_result = f_oneway(*grouped_data)
        anova_results_race[(analyte, race)] = anova_result

anova_results_race


In [ ]:
# Merge analyte data with the filtered data to include 'Age' information
merged_data = pd.merge(analyte_data_filtered, filtered_data[['subjectAccession', 'armName', 'gender', 'race', 'Age']], on='subjectAccession', how='left')

# Create a new column to split participants into age groups
merged_data['ageGroup'] = merged_data['Age'].apply(lambda x: 'Under 40' if x < 40 else '40 and above')

# Create box plots for each armName and antibody, split by age groups, for each day
fig, axes = plt.subplots(len(analytes_of_interest), len(merged_data['armName'].unique()), figsize=(24, 18), sharey=True)

for i, analyte in enumerate(analytes_of_interest):
    for j, armName in enumerate(merged_data['armName'].unique()):
        data_to_plot = merged_data[(merged_data['analyteReported'] == analyte) & (merged_data['armName'] == armName)]
        sns.boxplot(x='studyTimeCollectedDays', y='valueReported', hue='ageGroup', data=data_to_plot, ax=axes[i, j])
        axes[i, j].set_title(f'{analyte} - {armName}')
        axes[i, j].set_xlabel('Days')
        axes[i, j].set_ylabel('Value Reported')
        if i == len(analytes_of_interest) - 1:
            axes[i, j].set_xlabel('Days')
        else:
            axes[i, j].set_xlabel('')

plt.tight_layout()
plt.show()


In [ ]:
# Function to create heatmap data by age group
def create_heatmap_data_by_age_group(data, analyte, age_group):
    pivot_data = data[(data['analyteReported'] == analyte) & (data['ageGroup'] == age_group)].pivot_table(
        values='valueReported', 
        index='studyTimeCollectedDays', 
        columns='armName', 
        aggfunc='mean'
    )
    return pivot_data

age_groups = ['Under 40', '40 and above']

# Create heatmaps for each age group
fig, axes = plt.subplots(3, len(age_groups), figsize=(24, 18))

for i, analyte in enumerate(analytes_of_interest):
    for j, age_group in enumerate(age_groups):
        heatmap_data = create_heatmap_data_by_age_group(merged_data, analyte, age_group)
        sns.heatmap(heatmap_data, ax=axes[i, j], cmap='viridis', cbar=True)
        axes[i, j].set_title(f'{analyte} - {age_group}')
        axes[i, j].set_xlabel('armName')
        axes[i, j].set_ylabel('Days')

plt.tight_layout()
plt.show()


In [ ]:
# Prepare data for ANOVA by age group
anova_results_age_group = {}

for analyte in analytes_of_interest:
    for age_group in age_groups:
        analyte_data = merged_data[(merged_data['analyteReported'] == analyte) & (merged_data['ageGroup'] == age_group)]
        grouped_data = analyte_data.groupby('studyTimeCollectedDays')['valueReported'].apply(list)
        
        # Run ANOVA
        anova_result = f_oneway(*grouped_data)
        anova_results_age_group[(analyte, age_group)] = anova_result

anova_results_age_group


In [ ]:
# Perform Tukey's HSD test for IgA and IgM in the under 40 age group
tukey_result_IgA_under_40 = tukey_hsd_test(merged_data[(merged_data['analyteReported'] == 'IgA') & (merged_data['ageGroup'] == 'Under 40')], 'IgA')
tukey_result_IgM_under_40 = tukey_hsd_test(merged_data[(merged_data['analyteReported'] == 'IgM') & (merged_data['ageGroup'] == 'Under 40')], 'IgM')

# Extract and display significant comparisons for Tukey's HSD test for IgA and IgM in the under 40 age group
significant_IgA_under_40 = tukey_result_IgA_under_40.summary().data[1:]
significant_IgM_under_40 = tukey_result_IgM_under_40.summary().data[1:]

significant_IgA_under_40 = [row for row in significant_IgA_under_40 if row[-1] == 'True']
significant_IgM_under_40 = [row for row in significant_IgM_under_40 if row[-1] == 'True']

# Convert results to DataFrames for better readability
df_significant_IgA_under_40 = pd.DataFrame(significant_IgA_under_40, columns=tukey_result_IgA_under_40.summary().data[0])
df_significant_IgM_under_40 = pd.DataFrame(significant_IgM_under_40, columns=tukey_result_IgM_under_40.summary().data[0])


df_significant_IgA_under_40, df_significant_IgM_under_40
